# API Service 测试 Notebook

本 Notebook 用于测试租赁协商系统的 API Service 功能，包括:
- 创建协商会话
- WebSocket 实时通信测试
- 管理端点测试
- 错误处理验证

## 测试环境设置

In [1]:
import asyncio
import json
import requests
import websockets
from datetime import datetime
from typing import Dict, Any

# 配置
BASE_URL = "http://localhost:8000"
WS_URL = "ws://localhost:8000"

# 测试数据
test_session_id = None
test_participants = {}

print("环境配置完成 ✓")

环境配置完成 ✓


## 1. 健康检查和基础信息

In [2]:
# 检查 API 服务状态
def check_api_health():
    try:
        response = requests.get(f"{BASE_URL}/")
        if response.status_code == 200:
            data = response.json()
            print("✓ API 服务运行正常")
            print(f"版本: {data.get('version')}")
            print(f"功能: {', '.join(data.get('features', []))}")
            return True
        else:
            print(f"✗ API 服务异常: {response.status_code}")
            return False
    except Exception as e:
        print(f"✗ 连接失败: {str(e)}")
        return False

api_healthy = check_api_health()

✓ API 服务运行正常
版本: 2.0.0
功能: Multi-party negotiations, Real-time WebSocket communication, Market analyst integration, Session management, Analytics and monitoring


## 2. 获取演示数据

In [3]:
# 获取演示协商数据
def get_demo_data():
    try:
        response = requests.get(f"{BASE_URL}/admin/demo/negotiation-data")
        if response.status_code == 200:
            data = response.json()
            print("✓ 获取演示数据成功")
            print("\n=== 房产信息 ===")
            prop = data['property']
            print(f"地址: {prop['address']}")
            print(f"月租: £{prop['rent']}")
            print(f"卧室数: {prop['bedrooms']}")
            print(f"房产类型: {prop.get('property_type', 'N/A')}")
            print(f"房产描述: {prop.get('summary', 'N/A')}")
            
            print("\n=== 房东信息 ===")
            landlord = data['landlord']
            print(f"姓名: {landlord['name']}")
            print(f"房产数量: {landlord['properties_count']}")
            preferences = landlord.get('preferences', {})
            if preferences:
                print("房东偏好:")
                for key, value in preferences.items():
                    print(f"  - {key}: {value}")
            
            print("\n=== 租客信息 ===")
            tenants = data['tenants']
            print(f"租客数量: {len(tenants)}")
            for i, tenant in enumerate(tenants, 1):
                print(f"\n租客 {i}:")
                print(f"  姓名: {tenant['name']}")
                print(f"  预算: £{tenant['budget']}")
                print(f"  卧室需求: {tenant['bedrooms_needed']}")
                print(f"  学生身份: {'是' if tenant.get('is_student') else '否'}")
                print(f"  有宠物: {'是' if tenant.get('has_pets') else '否'}")
                print(f"  有担保人: {'是' if tenant.get('has_guarantor') else '否'}")
            
            # 显示市场背景信息
            if 'market_context' in data:
                print("\n=== 市场背景 ===")
                market = data['market_context']
                print(f"区域平均租金: £{market.get('average_rent_in_area', 'N/A')}")
                print(f"类似房产数量: {market.get('similar_properties_available', 'N/A')}")
                print(f"市场趋势: {market.get('market_trend', 'N/A')}")
            
            # 显示协商建议
            if 'negotiation_tips' in data:
                print("\n=== 协商建议 ===")
                tips = data['negotiation_tips']
                print(f"房东策略: {tips.get('landlord_strategy', 'N/A')}")
                print(f"租客策略: {tips.get('tenant_strategy', 'N/A')}")
                print(f"市场因素: {tips.get('market_factors', 'N/A')}")
            
            return data
        else:
            print(f"✗ 获取演示数据失败: {response.status_code}")
            if response.text:
                print(f"错误信息: {response.text}")
            return None
    except Exception as e:
        print(f"✗ 请求失败: {str(e)}")
        return None

demo_data = get_demo_data()
if demo_data:
    test_participants = {
        'landlord_id': demo_data['landlord']['id'],
        'tenant_ids': [t['id'] for t in demo_data['tenants']],
        'property_id': demo_data['property']['id']
    }
    print(f"\n✓ 测试参与者信息已设置")
    print(f"房东 ID: {test_participants['landlord_id']}")
    print(f"租客 ID: {test_participants['tenant_ids']}")
    print(f"房产 ID: {test_participants['property_id']}")
else:
    print("\n✗ 无法设置测试参与者 - 演示数据获取失败")

✓ 获取演示数据成功

=== 房产信息 ===
✗ 请求失败: 'property'

✗ 无法设置测试参与者 - 演示数据获取失败


## 3. 创建协商会话测试

In [4]:
# 创建协商会话
def create_negotiation_session():
    if not demo_data:
        print("✗ 需要先获取演示数据")
        return None
        
    payload = {
        "property_id": test_participants['property_id'],
        "tenant_ids": test_participants['tenant_ids'][:2],  # 取前2个租客
        "landlord_id": test_participants['landlord_id']
    }
    
    try:
        response = requests.post(
            f"{BASE_URL}/negotiation/create",
            json=payload,
            headers={"Content-Type": "application/json"}
        )
        
        if response.status_code == 200:
            data = response.json()
            session_id = data['session_id']
            print(f"✓ 协商会话创建成功: {session_id}")
            print(f"状态: {data['status']}")
            print(f"参与者数量: {len(data['participants'])}")
            
            for participant in data['participants']:
                print(f"  - {participant['name']} ({participant['role']})")
            
            return session_id
        else:
            print(f"✗ 创建失败: {response.status_code}")
            print(f"错误: {response.text}")
            return None
    except Exception as e:
        print(f"✗ 请求失败: {str(e)}")
        return None

test_session_id = create_negotiation_session()

✗ 需要先获取演示数据


## 4. 查询协商会话信息

In [5]:
# 查询会话详情
def get_session_info(session_id):
    if not session_id:
        print("✗ 需要有效的会话 ID")
        return None
        
    try:
        response = requests.get(f"{BASE_URL}/negotiation/{session_id}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"✓ 会话信息获取成功")
            print(f"会话 ID: {data['session_id']}")
            print(f"房产 ID: {data['property_id']}")
            print(f"状态: {data['status']}")
            print(f"创建时间: {data['created_at']}")
            print(f"更新时间: {data['updated_at']}")
            
            print("\n房产信息:")
            prop = data['context']['property']
            print(f"  地址: {prop['address']}")
            print(f"  月租: £{prop['monthly_rent']}")
            print(f"  卧室数: {prop['bedrooms']}")
            
            return data
        else:
            print(f"✗ 查询失败: {response.status_code}")
            return None
    except Exception as e:
        print(f"✗ 请求失败: {str(e)}")
        return None

session_info = get_session_info(test_session_id)

✗ 需要有效的会话 ID


## 5. WebSocket 实时通信测试

In [6]:
# WebSocket 测试函数
async def test_websocket_communication():
    if not test_session_id:
        print("✗ 需要有效的会话 ID")
        return
        
    ws_url = f"{WS_URL}/ws/negotiation/{test_session_id}"
    print(f"连接到: {ws_url}")
    
    try:
        async with websockets.connect(ws_url) as websocket:
            print("✓ WebSocket 连接成功")
            
            # 接收初始会话信息
            session_info = await websocket.recv()
            session_data = json.loads(session_info)
            print(f"会话信息: {session_data['type']}")
            
            # 模拟租客发送消息
            tenant_id = test_participants['tenant_ids'][0]
            test_message = {
                "message": "Hello, I'm interested in this property. Can we discuss the rent?",
                "participant_id": tenant_id
            }
            
            print(f"\n发送消息: {test_message['message']}")
            await websocket.send(json.dumps(test_message))
            
            # 接收响应
            response_count = 0
            full_response = ""
            
            while response_count < 10:  # 限制接收次数
                try:
                    response = await asyncio.wait_for(websocket.recv(), timeout=2.0)
                    data = json.loads(response)
                    
                    if data['type'] == 'message_received':
                        print(f"✓ 消息已接收: {data['from']} ({data['role']})")
                    elif data['type'] == 'response_chunk':
                        full_response += data.get('chunk', '')
                        print(f"接收响应块: {data.get('chunk', '')[:50]}...")
                    elif data['type'] == 'response_complete':
                        print(f"\n✓ 完整响应接收完成")
                        print(f"响应长度: {len(data.get('response', ''))} 字符")
                        break
                    elif data['type'] == 'error':
                        print(f"✗ 错误: {data.get('error')}")
                        break
                        
                    response_count += 1
                except asyncio.TimeoutError:
                    print("响应接收超时")
                    break
                    
            print(f"\n总共接收 {response_count} 个响应")
            
    except Exception as e:
        print(f"✗ WebSocket 测试失败: {str(e)}")

# 运行 WebSocket 测试
if test_session_id:
    await test_websocket_communication()
else:
    print("✗ 跳过 WebSocket 测试 - 无有效会话")

✗ 跳过 WebSocket 测试 - 无有效会话


## 6. 管理端点测试

In [7]:
# 测试管理端点
def test_admin_endpoints():
    print("=== 管理端点测试 ===")
    
    # 1. 健康检查
    try:
        response = requests.get(f"{BASE_URL}/admin/health")
        if response.status_code == 200:
            print("✓ 健康检查通过")
        else:
            print(f"✗ 健康检查失败: {response.status_code}")
    except Exception as e:
        print(f"✗ 健康检查异常: {str(e)}")
    
    # 2. 获取示例参与者
    try:
        response = requests.get(f"{BASE_URL}/admin/participants/sample?count=3")
        if response.status_code == 200:
            data = response.json()
            print(f"✓ 获取示例参与者成功 ({len(data.get('participants', []))} 个)")
        else:
            print(f"✗ 获取示例参与者失败: {response.status_code}")
    except Exception as e:
        print(f"✗ 获取示例参与者异常: {str(e)}")
    
    # 3. 会话统计
    try:
        response = requests.get(f"{BASE_URL}/admin/analytics/sessions")
        if response.status_code == 200:
            data = response.json()
            print(f"✓ 会话统计获取成功")
            print(f"  总会话数: {data.get('total_sessions', 0)}")
            print(f"  活跃会话数: {data.get('active_sessions', 0)}")
        else:
            print(f"✗ 会话统计失败: {response.status_code}")
    except Exception as e:
        print(f"✗ 会话统计异常: {str(e)}")
    
    # 4. 热门房产
    try:
        response = requests.get(f"{BASE_URL}/admin/analytics/popular-properties?limit=3")
        if response.status_code == 200:
            data = response.json()
            popular = data.get('popular_properties', [])
            print(f"✓ 热门房产获取成功 ({len(popular)} 个)")
            for prop in popular[:2]:  # 显示前2个
                print(f"  - {prop.get('address', 'Unknown')} (协商次数: {prop.get('negotiation_count', 0)})")
        else:
            print(f"✗ 热门房产失败: {response.status_code}")
    except Exception as e:
        print(f"✗ 热门房产异常: {str(e)}")

test_admin_endpoints()

=== 管理端点测试 ===
✓ 健康检查通过
✓ 获取示例参与者成功 (0 个)
✓ 会话统计获取成功
  总会话数: 0
  活跃会话数: 0
✓ 热门房产获取成功 (0 个)


## 7. 错误处理测试

In [8]:
# 错误场景测试
def test_error_scenarios():
    print("=== 错误处理测试 ===")
    
    # 1. 无效会话 ID
    try:
        response = requests.get(f"{BASE_URL}/negotiation/invalid-session-id")
        if response.status_code == 404:
            print("✓ 无效会话 ID 正确返回 404")
        else:
            print(f"✗ 无效会话 ID 返回意外状态码: {response.status_code}")
    except Exception as e:
        print(f"✗ 无效会话 ID 测试异常: {str(e)}")
    
    # 2. 创建协商时缺少参数
    try:
        incomplete_payload = {
            "property_id": "missing_other_fields"
        }
        response = requests.post(
            f"{BASE_URL}/negotiation/create",
            json=incomplete_payload,
            headers={"Content-Type": "application/json"}
        )
        if response.status_code == 422:  # Validation error
            print("✓ 缺少参数正确返回 422")
        else:
            print(f"✗ 缺少参数返回意外状态码: {response.status_code}")
    except Exception as e:
        print(f"✗ 缺少参数测试异常: {str(e)}")
    
    # 3. 不存在的参与者
    try:
        invalid_payload = {
            "property_id": "prop_123",
            "tenant_ids": ["nonexistent_tenant"],
            "landlord_id": "nonexistent_landlord"
        }
        response = requests.post(
            f"{BASE_URL}/negotiation/create",
            json=invalid_payload,
            headers={"Content-Type": "application/json"}
        )
        if response.status_code in [404, 500]:  # Not found or server error
            print("✓ 不存在的参与者正确返回错误状态")
        else:
            print(f"✗ 不存在的参与者返回意外状态码: {response.status_code}")
    except Exception as e:
        print(f"✗ 不存在的参与者测试异常: {str(e)}")

test_error_scenarios()

=== 错误处理测试 ===
✗ 无效会话 ID 返回意外状态码: 500
✓ 缺少参数正确返回 422
✓ 不存在的参与者正确返回错误状态


## 8. 性能测试

In [9]:
import time

# 简单性能测试
def performance_test():
    print("=== 性能测试 ===")
    
    # 测试根端点响应时间
    times = []
    for i in range(5):
        start_time = time.time()
        try:
            response = requests.get(f"{BASE_URL}/")
            end_time = time.time()
            if response.status_code == 200:
                response_time = (end_time - start_time) * 1000  # 转换为毫秒
                times.append(response_time)
                print(f"请求 {i+1}: {response_time:.2f}ms")
        except Exception as e:
            print(f"请求 {i+1} 失败: {str(e)}")
    
    if times:
        avg_time = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        print(f"\n响应时间统计:")
        print(f"  平均: {avg_time:.2f}ms")
        print(f"  最小: {min_time:.2f}ms")
        print(f"  最大: {max_time:.2f}ms")
        
        if avg_time < 100:
            print("✓ 响应时间良好 (<100ms)")
        elif avg_time < 500:
            print("⚠ 响应时间一般 (100-500ms)")
        else:
            print("✗ 响应时间较慢 (>500ms)")

performance_test()

=== 性能测试 ===
请求 1: 2.65ms
请求 2: 2.33ms
请求 3: 1.86ms
请求 4: 1.77ms
请求 5: 1.66ms

响应时间统计:
  平均: 2.05ms
  最小: 1.66ms
  最大: 2.65ms
✓ 响应时间良好 (<100ms)


## 9. 测试总结

In [10]:
# 生成测试报告
def generate_test_report():
    print("\n" + "="*50)
    print("           API SERVICE 测试报告")
    print("="*50)
    print(f"测试时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"测试环境: {BASE_URL}")
    print()
    
    # 测试结果统计
    print("测试结果概述:")
    print(f"✓ API 服务健康状态: {'正常' if api_healthy else '异常'}")
    print(f"✓ 演示数据获取: {'成功' if demo_data else '失败'}")
    print(f"✓ 协商会话创建: {'成功' if test_session_id else '失败'}")
    print(f"✓ 会话信息查询: {'成功' if session_info else '失败'}")
    print()
    
    if test_session_id:
        print(f"创建的测试会话 ID: {test_session_id}")
        print(f"参与者信息:")
        print(f"  - 房东 ID: {test_participants.get('landlord_id', 'N/A')}")
        print(f"  - 租客 ID: {test_participants.get('tenant_ids', [])}")
        print(f"  - 房产 ID: {test_participants.get('property_id', 'N/A')}")
    
    print()
    print("建议:")
    if not api_healthy:
        print("- 检查 API 服务是否正在运行")
        print("- 验证服务器配置和依赖")
    if not demo_data:
        print("- 检查数据库连接")
        print("- 确认演示数据已初始化")
    if api_healthy and demo_data:
        print("- 所有基础功能运行正常")
        print("- 可以进行更深入的功能测试")
        print("- 建议进行负载测试验证性能")

generate_test_report()


           API SERVICE 测试报告
测试时间: 2025-06-06 16:12:16
测试环境: http://localhost:8000

测试结果概述:
✓ API 服务健康状态: 正常
✓ 演示数据获取: 失败
✓ 协商会话创建: 失败
✓ 会话信息查询: 失败


建议:
- 检查数据库连接
- 确认演示数据已初始化


## 测试使用说明

### 前置条件
1. 确保 API 服务在 localhost:8000 运行
2. 数据库已连接并包含演示数据
3. 安装必要的 Python 包: requests, websockets

### 运行测试
1. 按顺序执行各个代码单元
2. 观察每个测试的输出结果
3. 根据错误信息进行故障排除

### 实际场景测试
- **创建协商会话**: 模拟房东和租客之间的协商场景
- **WebSocket 通信**: 测试实时消息传递
- **管理功能**: 验证系统监控和分析能力
- **错误处理**: 确保系统稳定性

### 参与者角色
- **房东**: 房产所有者，可以设置租金和条件
- **租客**: 潜在承租人，可以提出租金和条件建议

### 扩展测试
- 多并发用户测试
- 长时间连接稳定性测试
- 大量数据处理性能测试
- 异常场景恢复测试